In [19]:
#CODE FOR INITIALIZING REWARD TABLE
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os
from textblob import TextBlob
import pandas as pd
import pickle
import os

dir='C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'
strategies_dict=pickle.load( open("C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/strategies.pkl", "rb" ) )

N_TURN = 3
list1=[]
bot_strategies=[]
learning_rate=0.01
discount_factor=0.9
epsilon=0.9

user_utterances=[]
strategies_list=[]

evaluation_dict={}
user_lines=[]
#state action table
R_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q'ls-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

#temporary dictionary which stores the count of the recorded utterance till date,this is used for calculating the average to be stored in state table
temp_dict={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}

temp_dict1={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}
Q_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q'ls-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

In [20]:
#utility functions
def check_strategy(response):
    for key, value in strategies_dict.items():
        if response in value:
            #print(key)
            break
            
    return key
def check_strategies(response):
    temp_list=response.split(":")
    temp_list2=temp_list[0].split('_')
    return temp_list2[1]
    

def get_utterance(line):
    temp_list=line.split(":")
    value=temp_list[3]
    return value

def populate_evaluation(line):
    list1=line.split(',')
    for values in list1:
        if values is not '':
            list2=values.split('=')
            evaluation_dict[list2[0]]=list2[1]

#utility functions
def next_state(utterance_no):
    if not utterance_no==len(user_utterances)-1:
        return user_utterances[utterance_no+1]
    else:
        return 0
    
def max_q_value(utterance):
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
    return max

def training(indexes):
    
    for line in user_utterances:
        next_utterance=next_state(indexes)
        prev_value=Q_table.at[user_utterances[indexes],strategies_list[indexes]]
        if next_utterance is not 0:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=((temp_dict[user_utterances[indexes]]-1)*Q_table.at[user_utterances[indexes],strategies_list[indexes]]+R_table.at[user_utterances[indexes],strategies_list[indexes]] + discount_factor*max_q_value(next_utterance))/temp_dict[user_utterances[indexes]] 
        else:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=R_table.at[user_utterances[indexes],strategies_list[indexes]]

#def training_learning():
    

#def learn():
    

In [21]:
list1=[]
user_utterances=[]
bot_strategies=[]
#function which updates the state value table for each conversation where list1 is the utterance of the user and list2 is the strategy
#applied by the bot
def record_check(list1,list2,evaluated_score):
    i=0
    for key in list1:
        #print state_action_dict.at[key,list2[i]]
        previous_value=state_action_dict.at[key,list2[i]]
        temp_dict[key]=temp_dict[key]+1
        #print(float(evaluated_score))
        #print temp_dict[key],((previous_value*(temp_dict[key]-1))+evaluated_score)/temp_dict[key]
        value=((previous_value*(temp_dict[key]-1))+float(evaluated_score))/temp_dict[key]
        #print(value)
        state_action_dict.at[key,list2[i]]=value
        i=i+1
    

def rows_utterance(list1):
    
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if beginning_end(values,list1)=="b":
                    result="qs+b"
                else:
                    result="qs-e"
            elif sentiment(values)=="s-":
                if beginning_end(values,list1)=="b":
                    result="qs-b"
                else:
                    result="qs-e"
            else:
                if beginning_end(values,list1)=="b":
                    result="qs'b"
                else:
                    result="qs'e"
        else:
            if count_no_words(values)=="l":
            
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'ls+b"
                    else:
                        result="q'ls+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'ls-b"
                    else:
                        result="q'ls-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'ls'b"
                    else:
                        result="q'ls'e"
            else:
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'l's+b"
                    else:
                        result="q'l's+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'l's-b"
                    else:
                        result="q'l's-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'l's'b"
                    else:
                        result="q'l's'e"
        user_utterances.append(result)
    return user_utterances

def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    #print q
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        #print "long"
        l="l"
    else:
        #print "short"
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)==0:
        #print "beginning"
        var="b"
    else:
        #print "ending"
        var="e"
    return var

In [22]:
#CODE FOR EXTRACTING VALUES FROM REWARD TABLE AND LEARNING FROM IT

user_utterances=[]

for filename in os.listdir(dir):
    
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        #print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
                
        else:
            
            if lines.endswith(','):
                populate_evaluation(lines)
                user_utterances=rows_utterance(user_lines)
                temp_dict[user_utterances[0]]=temp_dict[user_utterances[0]]+1
                R_table.at[user_utterances[0],strategies_list[0]]=((temp_dict[user_utterances[0]]-1)*R_table.at[user_utterances[0],strategies_list[0]]+0.8*float(evaluation_dict['start'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[0]]
                count=1
                for line in user_utterances[1:len(user_utterances)]:
                    #print(count)
                    temp_dict[user_utterances[count]]=temp_dict[user_utterances[count]]+1
                    R_table.at[user_utterances[count],strategies_list[count]]=((temp_dict[user_utterances[count]]-1)*R_table.at[user_utterances[count],strategies_list[count]]+0.4*float(evaluation_dict['engaing'])+0.4*float(evaluation_dict['interupt'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[count]]               
                    
                    
                    count=count+1
            else:
                pass
        
    file.close()

In [23]:
R_table

,question,elaborate,opinion,affirmative,negative,neutral
qs+b,0.000000,0.200000,0.166667,0.510476,0.000000,0.283333
qs+e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-b,0.000000,0.150000,0.000000,0.000000,0.200000,0.000000
qs-e,0.105714,1.116293,0.801147,0.938065,0.795976,1.720955
qs'b,0.000000,0.151118,0.218139,0.238188,0.120572,0.076774
qs'e,0.068910,0.376016,0.660562,0.425230,1.293895,0.782824
q'ls+b,4.400000,0.000000,0.866667,0.000000,0.000000,0.000000
q'ls+e,0.580556,0.473913,1.872320,0.262500,0.888889,0.945714
q'ls-b,4.300000,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls-e,1.020000,1.159259,0.090909,0.000000,0.000000,0.250000


In [24]:
#for training from the model
for filename in os.listdir(dir):
    
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        #print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
    user_utterances=rows_utterance(user_lines)
    for values in user_utterances:
        indexes=user_utterances.index(values)
        training(indexes)

In [25]:
Q_table

,question,elaborate,opinion,affirmative,negative,neutral
qs+b,0.000000,0.729586,0.725808,2.831461,0.000000,2.406625
qs+e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-b,0.000000,1.855076,0.000000,0.000000,1.515415,0.000000
qs-e,0.300765,2.015153,1.581308,0.938065,2.488061,3.110319
qs'b,0.000000,1.918855,1.189928,2.029270,1.480820,1.416158
qs'e,0.478802,0.566682,1.959463,0.821659,1.983678,1.155335
q'ls+b,6.454917,0.000000,3.422184,0.000000,0.000000,0.000000
q'ls+e,0.580556,2.660772,3.650080,0.705870,2.039868,1.906371
q'ls-b,6.869013,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls-e,2.394139,3.314173,0.090909,0.000000,0.000000,0.730376
